## Sentiment Analysis using BERT pre-trained model

This notebook was developed while following [Coursera Tutorial](https://www.coursera.org/projects/sentiment-analysis-bert) on BERT. 

### Exploratory data Analysis

In [1]:
import pandas as pd

In [2]:
# reading and viewing the dataset
# because the dataset is just a collection of id, tweets and emotions with no column names,  
# we add names while reading in the data
df = pd.read_csv("Data/smile-annotations-final.csv", names = ['id', 'tweets', 'emotions'])

In [3]:
df.set_index('id', inplace=True)

In [4]:
df.head()

,tweets,emotions
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


,tweets,emotions
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [5]:
# to view all the different emotion categories, we do:

df.emotions.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: emotions, dtype: int64

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: emotions, dtype: int64

In [6]:
# to simplify the process, removing multilabel classification problem

df = df[~df.emotions.str.contains('\|')]

In [7]:
# can also removing nocode, because that doesn't add anything to the model:
df = df[df.emotions != 'nocode']

In [8]:
df.emotions.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: emotions, dtype: int64

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: emotions, dtype: int64

In [9]:
possible_emotions = df.emotions.unique()

In [10]:
# encoding the emotions
dict_emotions = {}
for index, emotion in enumerate(possible_emotions):
    dict_emotions[emotion] = index

In [11]:
df['emotion_label'] = df.emotions.replace(dict_emotions)

### Creating Training Data

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
# because test data needs to see all examples of training data and 
# there is imbalanced class problem, using stratified splitting over different emotions

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.emotion_label.values, 
                                                  test_size=0.15, 
                                                  random_state=17, 
                                                  stratify=df.emotion_label.values)

In [14]:
df['data_type'] = ['not_set']*df.shape[0]

In [15]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [16]:
df.groupby(['emotions', 'emotion_label', 'data_type']).count()

tweets
emotions     emotion_label data_type        
angry        2             train          48
                           val             9
disgust      3             train           5
                           val             1
happy        0             train         966
                           val           171
not-relevant 1             train         182
                           val            32
sad          4             train          27
                           val             5
surprise     5             train          30
                           val             5

tweets
emotions     emotion_label data_type        
angry        2             train          48
                           val             9
disgust      3             train           5
                           val             1
happy        0             train         966
                           val           171
not-relevant 1             train         182
                           val            32
sad          4             train          27
                           val             5
surprise     5             train          30
                           val             5

### Feature Engineering - Tokenizing and Encoding the Data

In [17]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
import torch

In [18]:
# we use the bert-base-uncased pretrained model.

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [19]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].tweets.values, 
    add_special_tokens=True, # required for signalling end of line.
    return_attention_mask=True, # required for using attention mechanism
    pad_to_max_length=True, # using a fixed max length
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].tweets.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].emotion_label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].emotion_label.values)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Users/sankrantijoshi/Documents/miniconda/envs/BERT/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1767: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or

In [20]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [21]:
len(dataset_train)

1258

1258

In [22]:
len(dataset_val)

223

223

### Setting up pre-trained model for training

In [23]:
from transformers import BertForSequenceClassification

In [24]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(dict_emotions),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertF

### Creating data loaders

In [25]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [26]:
batch_size = 4

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

### Setting up optimizer and scheduler

In [27]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [28]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [29]:
epochs = 1

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

### Defining Performance metrics

In [30]:
import numpy as np
from sklearn.metrics import f1_score

In [31]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [32]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in dict_emotions.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

### Creating Training Loop

In [33]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)

print(device)

cpu
cpu


In [35]:
from tqdm.notebook import tqdm

In [36]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [37]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.7626114481025272

Epoch 1
Training loss: 0.7626114481025272
Validation loss: 0.6481622198064413
F1 Score (Weighted): 0.7360109183076624

Validation loss: 0.6481622198064413
F1 Score (Weighted): 0.7360109183076624



In [38]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [39]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy: 170/171

Class: not-relevant
Accuracy: 8/32

Class: angry
Accuracy: 0/9

Class: disgust
Accuracy: 0/1

Class: sad
Accuracy: 0/5

Class: surprise
Accuracy: 0/5

Class: happy
Accuracy: 170/171

Class: not-relevant
Accuracy: 8/32

Class: angry
Accuracy: 0/9

Class: disgust
Accuracy: 0/1

Class: sad
Accuracy: 0/5

Class: surprise
Accuracy: 0/5

